In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

#### Imports for trees

In [ ]:
from sklearn import tree 
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier

from sklearn.datasets import load_breast_cancer

## Tree-based methods for Regression and Classification

* Non-parametric supervised learning methods for classification and regression.
* These methods learn simple decision rules inferred from the training data to predict target values
* Tree methods segment the feature space into regions based on Decision (Splitting) rules 
    - Trees are a representation of these rules 


In [ ]:
Hitters = pd.read_csv("Hitters.csv")
Hitters = Hitters.dropna()
Hitters = Hitters.drop(['League','Division','NewLeague'],axis = 1)
Hitters['LogSalary'] = np.log(Hitters.Salary)
Hitters.tail()

#### Tree

* Sequence of binary decisions
* Build Tree based on Training data
    * Decide which feature and feature value to split on
    * Root node: the top at the top of the tree
    * Leaf (terminal) node: Lowest node, contains value or class of target variable



![](simpleDecisionTree1.png)
$$\text{Figure 1. Simple Decision Tree}$$

1. Split on Years < 4.5
2. Split on Hits < 117.5
 

#### Divides the space into regions ($R_1, R_2, R_3, ...$)

In [ ]:
sns.scatterplot(x = 'Years',y='Hits',hue='LogSalary',data = Hitters)
plt.plot([4.5,4.5],[0,250])
plt.plot([4.5,25],[117.5,117.5])
plt.plot(3,100,color = 'black',marker = 's');


#### Prediction

* Predict new observation by transversing the tree from the top according to the features and feature values of the new data point.

* For new point (black square) assign value or class based on a metric of other points in the region (i.e. node)

### History


* CART (Breiman et. al., 1984): Classification and Regression Trees
    - CART constructs binary trees using the feature and threshold that yield the largest information gain at each node.
    * sklearn uses an optimized version of CART except **Independent variables can not be categorical** at this time
* Quinlan: ID3(1986), C4.5(1993), C5.0 
    - only Classification Trees


### Recursive Binary Splitting (a top-down greedy algorithm)

* Given feature vector $X \in R^p$, it is computationally infeasible to consider every possible partition of the feature space into regions.  

![](Regions_tree.png)
$$\text{Figure 2. Regions and Tree produced by Recursive Bianry Splitting}$$

* Top-down: begins at the top of the tree and then successively splits the predictor space
* Greedy: at each step of the tree-building process, the best split is made at that particular step
    - No look ahead to pick a split that will lead to a better tree in some future step.
* Select the predictor $X_j$ and the cutpoints s such that splitting the space into the regions {X|$X_j$ < s} and {X|$X_j$ >  s} leads to the greatest possible reduction in some Regression or Classification metric
* Repeat the process on the resulting regions
    - Find the best $X_j$ and s to further split the data minimizing the Regression or Classification metric within each of the resulting regions. 
* Continue the process until a stopping criterion is reached
* Can use a feature multiple times with different values.



## Regression Trees

* Data: $(x_1,y_1),(x_2,y_2),...(x_n,y_n)$, $x_i \in R^d$, $y_i \in R$
    - d features
 
* Goal: Find regions $R_1,R_2,...,R_j$ to minimize Mean Squared Error (MSE) or Mean Absolute Error (MAE)
    - Which of the d features to split on and which value of the feature?

#### Regression Prediction

* Response of new observation is the mean of the training observations in the region to which the new observation belongs

### Regression Metrics

* Given Node m representing a region $R_m$ with $N_m$ observations ($x_i,y_i)$  
* The mean response for the training observations within the ith region is:

$$\bar{y} = \frac{1}{N_m}\sum_{i\in{N_m}}y_i$$

* $X_m$ is the training data in node m

#### $H(X_m)$ is called the impurity function.
* We want nodes to be pure

#### Mean Squared Error

* Minimizes L2 Error 

$$H(X_m) = \frac{1}{N_m}\sum_{i \in {N_m}}(y_i - \bar{y}_m)^2 $$

#### Mean Absolute Error

* Minimizes L1 Error 

$$H(X_m) = \frac{1}{N_m}\sum_{i \in {N_m}}|y_i - \bar{y}_m| $$
 

###  Overfitting   

* Decision Tree learning tends to make overly complex models which overfit
* Reduce size of tree since a smaller tree will have less variance 
    - Limit the depth of the tree
    - Set the minimum number of samples required at a leaf node
    - sklearn uses Minimal Cost-Complexity (see Appendix)
    

### Fitting Regression Trees in sklearn

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

#### Parameters

* criterion - the function to measure the quality of the split
    - 'mse'
    - 'friedman_mse'
    - 'mae'

* max_depth: control size by limiting tree depth
* min_samples_split: minimum # of samples required to split an internal node
* min_samples_leaf: control size by setting minimum number of samples at leaf nodes
* random_state: To obtain a deterministic behavior during fitting, set random_state.
* ccp_alpha: cost-complexity parameter used for Minimal Cost-Complexity Pruning. The subtree with the largest cost complexity that is smaller than ccp_alpha will be chosen

In [ ]:
#feats = ['Years','Hits']
feats = Hitters.columns.tolist()[:-2] # All the features
X = Hitters.loc[:,feats].values
y = Hitters.loc[:,'Salary'].values
X.shape,y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# Fit Tree to Training data
dt = DecisionTreeRegressor(random_state = 1234)
dt.fit(X_train, y_train)

In [ ]:
# Predict the Test data
y_pred = dt.predict(X_test)
mse = np.mean((y_pred - y_test)**2)
print("Test error is: ",mse)

In [ ]:
R2 = dt.score(X_test,y_test);R2
print(f'R-squared = {np.round(R2,3)}')

In [ ]:

vals = dt.feature_importances_
pairs = [(vals[i],feats[i]) for i in range(len(vals))]
pairs.sort(reverse=True)
for val,feat in pairs:
 print(feat,'\t',round(val,6))


In [ ]:
tree.plot_tree(dt);

#### 10-fold Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(dt, X,y, cv=10)
print(scores)
print(np.mean(scores))
plt.plot(range(1,11),scores);

#### Visualize Tree

In [ ]:

dot_data = tree.export_graphviz(dt, out_file=None, 
                         feature_names=feats,  
                         class_names=['Salary'],  
                         filled=True, rounded=True,  
                         special_characters=True) 

#### Install graphviz with conda or pip

In [ ]:
# pip install graphviz
# conda install graphviz
# conda install python-graphviz

import graphviz
graph = graphviz.Source(dot_data) 

In [ ]:
graph.render("Hitters",view = True,cleanup=True)

#### Save text dot file

In [ ]:
dotfile = open("Hitters_tree.dot", 'w')
tree.export_graphviz(dt, out_file=dotfile, feature_names=feats)
dotfile.close()

## Classification Trees
 
* Goal:  
    - Predict that each observation belongs to the most commonly occurring class of training observations in the region to which it belongs.
    
* Classes should be balanced to avoid  creating biased trees 


#### Predictions

* Response of new observation is the class the has the high proportion of observations in the region to which the new observation belongs

### Classification Metrics 

* Given Node m representing a region $R_m$ with $N_m$ observations ($x_i,y_i)$  
* The proportion if class k observations in node m is:

$$ p_{mk} = \frac{1}{N_m}\sum_{x_i \in R_m}I(y_i ==k)$$

* $X_m$ is the training data for node m

#### Gini impurity


    
$$H(X_m) = \sum_{k=1}^Kp_{mk}(1-p_{mk})$$
 
* where $p_{mk}$ is the proportion of training observations in the mth region that are from the kth class.
    - Takes on a small value if all of the $p_{mk}$s are close to
zero or one.
    - Node purity: a small value indicates that a node contains predominantly observations from a single class.
 
#### Entropy

* See Appendix
 
$$H(X_m) = -\sum_{k=1}^Kp_{mk}log(p_{mk})$$

* In most cases the Gini and Entropy metrics produce the same results

#### Misclassification

$$H(X_m) = 1 - max(p_{mk})$$

* Not widely used because not sensitive enough

### Fitting Classification Trees in sklearn

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier

#### Parameters

* criterion - the function to measure the quality of the split
    - 'gini'
    - 'entropy'

* max_depth: control size by limiting tree depth
* min_samples_split: minimum # of samples required to split an internal node
* min_samples_leaf: control size by setting minimum number of samples at leaf nodes
* random_state: To obtain a deterministic behavior during fitting, set random_state.
* ccp_alpha: cost-complexity parameter used for Minimal Cost-Complexity Pruning. The subtree with the largest cost complexity that is smaller than ccp_alpha will be chosen

In [ ]:
Wine = pd.read_csv("Wine.csv")
feats = Wine.columns[0:-1]
print(len(feats))
Wine.head()

In [ ]:
X = Wine.iloc[:,0:13].values
y = Wine.iloc[:,13].values


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify = y, test_size = 0.25, random_state = 1234)

In [ ]:
# Build decision tree classifier
dt = DecisionTreeClassifier(criterion='entropy')
dt.fit(X_train, y_train)

In [ ]:
# Predict the Test data
y_pred = dt.predict(X_test)

# Make  the Confusion Matrix and calculate  model accuracy

cm = confusion_matrix(y_test, y_pred)
print("Accuracy: ",round(np.trace(cm)/np.sum(cm),3))
cm

#### 10-fold Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score
np.random.seed(42)
scores = cross_val_score(dt, X,y, cv=10, scoring = 'accuracy')
print(scores)
print("Mean Accuracy: ",round(np.mean(scores),3))
plt.plot(range(1,11),scores);

In [ ]:
dt.n_features_

In [ ]:
dot_data = tree.export_graphviz(dt, out_file=None, 
                         feature_names= feats,  
                         class_names= Wine.columns[-1],  
                         filled=True, rounded=True,  
                         special_characters=True) 

In [ ]:
# pip install graphviz
# conda install graphviz
# conda install python-graphviz

import graphviz
graph = graphviz.Source(dot_data) 

graph.render("Wine",view = True,cleanup=True)

In [ ]:
dotfile = open("Wine_tree.dot", 'w')
tree.export_graphviz(dt, out_file=dotfile, feature_names=feats)
dotfile.close()

### Pros

* Simple and interpretable. 
* Trees can be visualized  
* Requires little data preparation (e.g. no scaling)
* Rules for numerical, ordinal and categorical data
* Mirror human decision-making??   
 

### Cons
 
* Tends to overfit. Creates overly complex trees that do not generalize well. 
    - Need to prune trees 
* Highly variable. Small change in data can produce completely different tree
* Finding optimum tree is NP-Complete.
    - Use Greedy algorithm which may not produce the best tree 
* Prediction accuracy is not competitive with other approaches
    - But with Ensemble Methods (Bagging, Random Forests and Boosting) can produce results that surpass other common methods in use today


## Pruning decision trees with cost complexity pruning (CCP)

* Cost Complexity Pruning controls the size of a tree to alleviate overfitting

* The keyword ccp_alpha is the CCP parameter.
    - Greater values increase number of nodes pruned
    
* The best value of ccp_alpha can be determined from accuracy scores

### Total impurity of leaves and alphas of pruned tree

* Minimal cost complexity pruning recursively finds the node with the "weakest
link". The weakest link is characterized by an effective alpha, where the
nodes with the smallest alpha are pruned first. 

* scikit-learn provides DecisionTreeClassifier.cost_complexity_pruning_path to 
determine the best  ccp_alpha.
    - It returns the alphas and the corresponding total leaf impurities at each step of
the pruning process. As alpha increases, more of the tree is pruned, which increases the total impurity of its leaves.



In [ ]:
data = load_breast_cancer(as_frame=True)
data.keys()

In [ ]:
df = data['frame']
df.tail()

In [ ]:
X = data['data'].values
y = data['target'].values
X.shape,y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.25,random_state=0,stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
model = DecisionTreeClassifier(random_state=0)
model.get_params()

In [ ]:
path = model.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

print(path.keys())
print(ccp_alphas.shape,impurities.shape)
ccp_alphas

* The maximum effective alpha value is removed, because
it is the trivial tree with only one node.



In [ ]:
fig, ax = plt.subplots()
ax.plot(ccp_alphas[:-1], impurities[:-1], marker='o', drawstyle="steps-post")
ax.set_xlabel("effective alpha")
ax.set_ylabel("total impurity of leaves")
ax.set_title("Total Impurity vs effective alpha for training set");

#### Train a decision tree using the effective alphas. 

* The last value in ccp_alphas is the alpha value that prunes the whole tree, leaving the last tree with one node.



In [ ]:
models = []
for ccp_alpha in ccp_alphas:
    model = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha)
    model.fit(X_train, y_train)
    models.append(model)
print(f"Number of nodes in the last tree is: {models[-1].tree_.node_count} with ccp_alpha: {ccp_alphas[-1]}")

####  Show that the number of nodes and tree depth decreases as alpha increases.



In [ ]:
models = models[:-1] # Remove lat element because it is the trivial tree with only one node
ccp_alphas = ccp_alphas[:-1]

node_counts = [m.tree_.node_count for m in models]
depth = [m.tree_.max_depth for m in models]
fig, ax = plt.subplots(2, 1)
ax[0].plot(ccp_alphas, node_counts, marker='o', drawstyle="steps-post")
ax[0].set_xlabel("alpha")
ax[0].set_ylabel("number of nodes")
ax[0].set_title("Number of nodes vs alpha")
ax[1].plot(ccp_alphas, depth, marker='o', drawstyle="steps-post")
ax[1].set_xlabel("alpha")
ax[1].set_ylabel("depth of tree")
ax[1].set_title("Depth vs alpha")
fig.tight_layout()

### Accuracy vs alpha for training and testing sets

* When ccp_alpha is set to zero and the otherparameters are set to their default values
the tree overfits
    - 100% training accuracy and 90% testing accuracy. 
* As alpha increases, more of the tree is pruned which will generalize better.


In [ ]:
train_scores = [m.score(X_train, y_train) for m in models]
test_scores = [m.score(X_test, y_test) for m in models]

fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")
ax.set_title("Accuracy vs alpha for training and testing sets")
ax.plot(ccp_alphas, train_scores, marker='o', label="train",
        drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker='o', label="test",
        drawstyle="steps-post")
ax.legend()
print(f'The best alpha is {ccp_alphas[np.argmax(test_scores)]}')

### Reference

https://scikit-learn.org/stable/auto_examples/tree/plot_cost_complexity_pruning.html#sphx-glr-auto-examples-tree-plot-cost-complexity-pruning-py

### Exercises

####  Pruning Decision Tree Regression

1) Determine the best ccp_alpha for the seaborn 'mpg' dataset.

2) Fit a Decision Tree Regressor to the seaborn 'mpg' dataset. Predict tip as a  function of all the features.  Use the ccp_alpha from 1)

3) Output feature importances and R-squared.

In [ ]:
mpg = sns.load_dataset('mpg')
mpg.tail()

In [ ]:
mpg.dropna(inplace=True)
np.sum(mpg.isna())

In [ ]:
X = mpg.iloc[:,1:-2].values
y = mpg.loc[:,'mpg'].values

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.20, random_state = 1234)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# Your Code Here

## Appendixes

### Information Entropy
 
#### Information: The reduction in uncertainty derived from learning an outcome

* Quantified by  minus the log probability of an event

<div style="font-size: 125%;">
$$ I = -log_2P(x)$$
</div>

* x is an event, e.g. a coin flip came up heads

In [ ]:
# Suppose P(X=heads) = 1/2, P(X = tails) = 1/2
from math import log

I = -log(1/2,2) #Base 2
print("{} bit of information".format(I))

In [ ]:
# Suppose P(X=heads) = 3/4, P(X = tails) = 1/4
I = -log(3/4,2)
print("{} bits of information".format(I))

P(x) = 3/4 > P(x) = 1/2, but I = -log(1/2,2) < I = -log(3/4,2)

Probability $\uparrow$ Information $\downarrow$


#### Information Entropy is the Average Information

* Minus the Expected Value of Information

* Suppose you have a 4-sided die with the probability of tossing a 1 = 1/2, a 2 = 1/4, 3 = 1/8, 4 =  1/8
    - i.e. pmf = (1/2,1/4,1/8,1/8)
    - E(X) = $\sum_ixp(x)$ = 1(1/2)+2(1/4)+3(1/8)+4(1/8)


In [ ]:
print("Expected value of X is: ", 1*(1/2)+2*(1/4)+3*(1/8)+4*(1/8))

<div style="font-size: 115%;">
$$ Entropy = H = -E(I) = -\sum_i p(x_i)*log(p(x_i))$$
</div>
* $p(x_i)$ is a PMF
  

In [ ]:
print("Entropy is: ",-1*(1/2*log(1/2,2) + 1/4*log(1/4,2) + 1/8*log(1/8,2) + 1/8*log(1/8,2)))

#### Differential Entropy - Continuous RV

<div style="font-size: 115%;">
$$Entropy = H = -\int{p(x)*log(p(x)} dx$$
</div>

* $p(x)$ is a PDF

### Minimal Cost-Complexity Pruning 

* Minimal cost-complexity pruning is an algorithm used to prune a tree to avoid over-fitting

* Minimal cost complexity pruning recursively finds the node with the "weakest
link" where the weakest link is characterized by the complexity parameter $\alpha \ge 0$ 
    - The nodes with the smallest alpha are pruned first. 
    
#### Cost-complexity measure $R_{\alpha}(T)$ of a given tree T

$$R_{\alpha}(T) = R(T) + \alpha|T|$$

|T|: number of terminal nodes in T  
R(T): total sample impurity of the terminal nodes in T  

*  Minimal cost-complexity pruning finds the subtree of T that minimizes $R_{\alpha}(T)$
    
#### Effective $\alpha$

* The cost complexity measure of a single node is $R_{\alpha}(t) = R(t) + \alpha$
* The branch $T_t$ is defined to be a tree where node t is the root node.
* The **effective $\alpha$** of a node t is when  $R_{\alpha}(T_t) = R_{\alpha}(t)$ i.e. when cost complexity measure of a node, t , and its branch $T_t$ are equal
$$ \alpha_{eff} = \frac{R(t) - R(T_t)}{|T|-1}$$

* A non-terminal node with the smallest value of $\alpha_{eff}$ is the weakest link and will be pruned. 
* Pruning stops when the tree’s minimal $\alpha_{eff}$ is greater than the ccp_alpha parameter in sklearn.



Figures 1 and 2 are taken from "An Introduction to Statistical Learning, with applications in R" (Springer, 2013) with permission from the authors: G. James, D. Witten, T. Hastie and R. Tibshirani "